In [1]:
import pandas as pd
import flywheel

In [42]:
from fw_heudiconv.cli.export import get_nested

200610-00:36:13,813 nipype.utils INFO:
	 Running nipype version 1.4.0 (latest: 1.5.0)


/storage/ttapera/fw-heudiconv/fw_heudiconv/backend_funcs/query.py:4: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  from nibabel.nicom.dicomwrappers import wrapper_from_data
INFO:nipype.utils:Running nipype version 1.4.0 (latest: 1.5.0)


In [17]:
client = flywheel.Client()

proj = client.projects.find_first('label="PNC_CS_810336"')
subjects = proj.subjects()

/home/ttapera/miniconda3/envs/flywheel/lib/python3.7/site-packages/flywheel/flywheel.py:6186: UserWarning: Client version 12.0.0 does not match server version 11.3.4. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


In [7]:
complete = pd.read_csv('/storage/ttapera/RBC/data/completed.txt')

In [9]:
complete = complete.drop_duplicates(subset=['subject'])
complete

,subject,download_ok,check_ok,date
0,1654508502,0,0,Fri May 29 17:26:05 EDT 2020
1,1344101127,0,0,Fri May 29 17:26:12 EDT 2020
2,1818459775,0,0,Fri May 29 17:26:16 EDT 2020
3,2748427955,0,0,Fri May 29 17:26:45 EDT 2020
4,278814796,0,0,Fri May 29 17:32:43 EDT 2020
...,...,...,...,...
2000,3810009389,0,0,Wed Jun 3 16:38:37 EDT 2020
2001,1919055552,0,0,Wed Jun 3 16:38:37 EDT 2020
2002,3671045126,0,0,Wed Jun 3 16:40:37 EDT 2020
2003,1762102677,1,0,Wed Jun 3 21:14:14 EDT 2020


Which subjects did not have a full download?

In [10]:
bad_download = complete[complete['download_ok'] == 1]

In [11]:
bad_download

,subject,download_ok,check_ok,date
5,1867827374,1,0,Fri May 29 17:33:23 EDT 2020
28,1604003015,1,0,Fri May 29 17:59:21 EDT 2020
33,2861717830,1,0,Fri May 29 18:06:24 EDT 2020
39,1103118018,1,0,Fri May 29 18:16:52 EDT 2020
40,2441231201,1,0,Fri May 29 18:17:00 EDT 2020
...,...,...,...,...
1991,2038321583,1,0,Wed Jun 3 16:30:55 EDT 2020
1993,1012530688,1,1,Wed Jun 3 16:31:41 EDT 2020
1997,1939119029,1,0,Wed Jun 3 16:36:05 EDT 2020
2003,1762102677,1,0,Wed Jun 3 21:14:14 EDT 2020


In [6]:
ids = pd.read_csv('/storage/ttapera/RBC/data/bblids_3.csv', header=None, names=['bblid', 'hash', 'full_id', 'process'])

In [12]:
ids

,bblid,hash,full_id,process
0,80010,1832999514,sub-1832999514,True
1,80179,2216595430,sub-2216595430,True
2,80199,1867827374,sub-1867827374,True
3,80208,278814796,sub-278814796,True
4,80249,3103368139,sub-3103368139,True
...,...,...,...,...
1596,139490,1012530688,sub-1012530688,True
1597,139496,3726299659,sub-3726299659,True
1598,139532,1919055552,sub-1919055552,True
1599,139538,3810009389,sub-3810009389,True


In [15]:
bad_download2 = bad_download.merge(ids[['bblid', 'hash']], left_on='subject', right_on='hash')

In [16]:
bad_download2

,subject,download_ok,check_ok,date,bblid,hash
0,1867827374,1,0,Fri May 29 17:33:23 EDT 2020,80199,1867827374
1,1604003015,1,0,Fri May 29 17:59:21 EDT 2020,81231,1604003015
2,2861717830,1,0,Fri May 29 18:06:24 EDT 2020,81353,2861717830
3,1103118018,1,0,Fri May 29 18:16:52 EDT 2020,81533,1103118018
4,2441231201,1,0,Fri May 29 18:17:00 EDT 2020,81604,2441231201
...,...,...,...,...,...,...
257,2038321583,1,0,Wed Jun 3 16:30:55 EDT 2020,139427,2038321583
258,1012530688,1,1,Wed Jun 3 16:31:41 EDT 2020,139490,1012530688
259,1939119029,1,0,Wed Jun 3 16:36:05 EDT 2020,139469,1939119029
260,1762102677,1,0,Wed Jun 3 21:14:14 EDT 2020,127611,1762102677


In [18]:
row = bad_download2.iloc[0,]

In [19]:
row

subject                          1867827374
download_ok                               1
check_ok                                  0
date           Fri May 29 17:33:23 EDT 2020
bblid                                 80199
hash                             1867827374
Name: 0, dtype: object

In [35]:
subject = [x for x in subjects if x.label == str(row['bblid'])][0]
session = subject.sessions()[0]


In [56]:
dtypes = []
acquisitions = session.acquisitions()

for acq in acquisitions:

    for f in acq.files:
        if f.type != 'nifti':
            continue
        dtypes.append(get_nested(f, 'info', 'BIDS', 'Folder'))
    
print(set(dtypes))

{'fmap', 'asl', 'dwi', 'anat', 'func', None}


In [58]:
all([x in dtypes for x in ['fmap', 'func', 'anat']])

True

In [59]:
def check_for_completeness(row, checks):
    
    subject = [x for x in subjects if x.label == str(row['bblid'])][0]
    session = subject.sessions()[0]
    dtypes = []
    acquisitions = session.acquisitions()
    for acq in acquisitions:

        for f in acq.files:
            if f.type != 'nifti':
                continue
            dtypes.append(get_nested(f, 'info', 'BIDS', 'Folder'))
    
    return [x in dtypes for x in checks]


In [61]:
check_for_completeness(row, ['fmap', 'func', 'anat', 'dwi'])

[True, True, True, True]